In [ ]:
#Import Packages
import pandas as pd
import numpy as np
import random
import re

In [1]:
#Load Data
X = pd.read_csv('./Data/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')

#Prompt ID
ASAPsets = [2]

#Test Set Selection
sampleSize = 50
seed = 27

In [ ]:
#Pre-process
X = X.rename(columns={"essay": "Text", "essay_set": "Set"})
X = X.loc[X['Set'].isin(ASAPsets)]

X = X.dropna(axis=1)
X['Index'] = np.arange(0, len(X))
X['Grade'] = X['domain1_score'] + X['domain2_score']
X['Text'] = X['Text'].str.replace(r'\.+', ". ",)
X['Text'] = X['Text'].str.replace(r'\ +', " ",)

X = X[['Index', 'Text', 'Grade']]
X['Grade'] = X['Grade'].apply(lambda x: x/10)
X = X.reset_index(drop=True)

#Add variables
for index, row in X.iterrows():
    essay = row['Text']
    grade = row['Grade']
    sentences = re.split('\? |\! |\.', essay)[:-1]
    words = essay.split(' ')
    X.at[index, 'sentences'] = len(sentences)
    X.at[index, 'words'] = len(essay)

In [ ]:
#Produce Train/Test split
np.random.seed(seed)
test = X.loc[np.random.choice(X.index, size=sampleSize, replace=False)]
X.drop(test.index, inplace=True)
train = X
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)

train.to_csv('./Data/train.csv', index=False)
test.to_csv('./Data/test.csv', index=False)